### Setup

- pip install dash pandas lxml nb_black dash-bootstrap-components
- download card_list.csv from 17Lands and save it in `data/` https://17lands-public.s3.amazonaws.com/analysis_data/cards/card_list.csv
- start this jupyter notebook with `jupyter notebook` from this path
- update the path to your log file (can be found in 17Lands client - may not be necessary)
- go to `localhost:8050/` to see the app!

In [ ]:
%load_ext nb_black

In [ ]:
import requests
import os
import json
import logging
from typing import List, Sequence, Union, Optional, Dict, Any, Tuple
import datetime
import time
from pathlib import Path

import plotly
import dash
from dash import dcc
from dash import html
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output
from dash import dash_table
from dash.dash_table.Format import Format, Scheme
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
from flask import Flask

from mtgradient.log_parsing.follower_logic import (
    DashFollower,
    ALSATrackerType,
    ColorTrackerType,
    get_config,
)

In [ ]:
def get_card_ratings_df(
    expansion: str,
    start_date="2020-01-01",
    end_date: Optional[str] = None,
    format_priority: List[str] = [
        "PremierDraft",
        "QuickDraft",
        "TradDraft",
        "CompDraft",
    ],
):
    if end_date is None:
        end_date = datetime.date.today().strftime("%Y-%m-%d")

    col_map = {
        "avg_seen": "alsa",
        "avg_pick": "ata",
        "opening_hand_win_rate": "oh wr",
        "ever_drawn_win_rate": "gd wr",
        "drawn_improvement_win_rate": "iwd",
    }
    card_df = None
    for format_str in format_priority:
        format_url = f"https://www.17lands.com/card_ratings/data?expansion={expansion.upper()}&format={format_str}&start_date={start_date}&end_date={end_date}"
        resp = requests.get(format_url)
        try:
            json_data = resp.json()
            card_df = pd.DataFrame(json_data)
        except (json.JSONDecodeError, ValueError) as e:
            print(f"failed fetching data for {format_url}")
            card_df = pd.DataFrame([])
        if len(card_df) == 0:
            print(
                f"failed to find data for {format_str} for {expansion} between {start_date} and {end_date}, trying next format."
            )
            time.sleep(1)  # try to be polite
        else:
            print(f"fetched {card_df.shape[0]} rows for {expansion} {format_str}")
            break
    return card_df.rename(columns=col_map)

def read_data(
    data_path: str,
    supported_expansions: List[str] = ["ZNR", "KHM", "STX", "AFR", "MID", "VOW"],
    card_ratings_kwargs: Dict[str, Any] = {},
    refresh=True,
) -> pd.DataFrame:
    """
    Programmatically pull the 17Lands card ratings and stack them. Also
    load the card_id mapping and join it.
    
    See get_card_ratings_df() for options  that can be passed to card_ratings_kwargs
    """
    if not refresh:
        return pd.read_csv("data/card_ratings_data.csv", index_col="id")
    card_ids_df = pd.read_csv(
        os.path.join(data_path, "card_list.csv"),
        usecols=[
            "id",
            "expansion",
            "name",
            "rarity",
            "color_identity",
            "mana_value",
            "types",
        ],
    )
    card_ids_df = card_ids_df[card_ids_df.expansion.isin(supported_expansions)]
    card_ids_df = card_ids_df[["id", "expansion", "name", "types"]]

    ratings_df = None
    for set_name in supported_expansions:
        single_df = get_card_ratings_df(set_name, **card_ratings_kwargs)

        single_df["expansion"] = set_name.upper()
        if ratings_df is None:
            ratings_df = single_df
        else:
            ratings_df = ratings_df.append(single_df)
        time.sleep(1) # try to be polite
    ratings_df = ratings_df.rename(columns={k: k.lower() for k in ratings_df.columns})

    joined = pd.merge(
        ratings_df,
        card_ids_df,
        how="left",
        left_on=["name", "expansion"],
        right_on=["name", "expansion"],
        suffixes=["", "_y"],
    )
    joined.to_csv("data/card_ratings_data.csv", index=False)
    return joined.set_index("id")


In [ ]:
# The first time you run, this needs to be True - in future runs, this will take extra time,
# but setting to True will pull all the 17Lands data you need freshly each time.
# setting to False will save time if you're okay with how recent your data is.
REFRESH_DATA = False
df = read_data("./data/", refresh=REFRESH_DATA)

In [ ]:
# so you can see available column names
df.head(5)

In [ ]:
# number of seconds between refreshes. Needs to be long enough to fully parse your
# .log file. If your file is huge, this could be an issue, and you may want to exit
# and reload the game
N_SECONDS = 10

# threshold for (pick_num - ALSA) at which we consider a
# card to be signifying some amount of "openness" of that
# color in the draft
LATE_ALSA_DIFF_THRESHOLD = 0.5

# if we see cards that show up "late" that are multicolor,
# how much should we weight this as an indicator that
# each of their colors is open?
MULTICOLOR_ALSA_DIFF_FACTOR = 0.5

# this parameter should be 0, but can be set to larger values to
# accommodate large logs. If you've played a lot and have many
# MB of data in your log, you might be better off skipping
# a lot of it. This will skip the first SEEK_TO bytes of your
# log. SET AT YOUR OWN RISK

# SEEK_TO = 32e6  skips 32MB
# SEEK_TO = 5e6  # skips 5MB
SEEK_TO = 0

# This is the path to your log file. If you want to test it out, I recommend
# saving a copy of one and editing it to have however many events you'd like
logfile = os.path.join(
    Path.home(),
    r"AppData\LocalLow\Wizards Of The Coast\MTGA\Player.log",
)

print(f"looking for logfile at: {logfile}")


# This may pop up a dialog the first time, I'm not sure.
token = get_config()

# This controls what displays. you can print(df) to see your column options
useful_cols = [
    "name",
    "color",
    "types",
    "rarity",
    "alsa",
    "ata",
    "oh wr",
    "gd wr",
    "iwd",
]

# this controls formatting of columns
percentage_cols = ["iwd", "oh wr", "gd wr"]
numeric_cols = ["alsa", "ata"]

# allows you to override the CSS of the dash app
external_stylesheets = ["https://codepen.io/chriddyp/pen/bWLwgP.css", dbc.themes.GRID]

# modify this to change the cell styling in the tables
cell_styling = {"textAlign": "left", "padding": "5px"}

# making server explicit so that eventually a Follower object could
# run separately and just post relevant events to an event API
# flask_app = Flask(__name__)

# the core of the app
app = dash.Dash(
    __name__,
    #     server=flask_app,
    #     url_base_pathname="/dash/",
    external_stylesheets=external_stylesheets,
)
app.layout = html.Div(
    html.Div(
        [
            dbc.Container(
                [
                    dbc.Row(html.H4("Current Options")),
                    dbc.Row(html.Div(id="live-update-text")),
                    dbc.Row(html.Div(id="current-pick-table-holder")),
                    dbc.Row(html.H4("Draft Info")),
                    dbc.Row(html.Div(id="draft-tracking-info")),
                    dbc.Row(html.H4("Pool So Far")),
                    dbc.Row(html.Div(id="pool-table-holder")),
                ]
            ),
            dcc.Interval(
                id="interval-component",
                interval=N_SECONDS * 1000,  # in milliseconds
                n_intervals=0,
            ),
        ]
    )
)
# init draft-level trackers that live outside the Follower
local_alsa_tracker: ALSATrackerType = {}
local_color_tracker: ColorTrackerType = {}


def get_color_count_graph(inp_df: pd.DataFrame) -> plotly.graph_objects.Figure:
    """
    Count the occurences of each "color" symbol in the card data's "color" column
    in the pool of cards you've drafted so far.
    """
    color_counts = (
        inp_df.color.apply(lambda x: list(x) if isinstance(x, str) else [])
        .explode()
        .value_counts()
        .reset_index()
        .rename(columns={"index": "Color", "color": "Count"})
    )
    color_hexes = ["#838383", "#26b569", "#f85656", "#aae0fa", "#fef2be"]
    missing_colors = [
        c for c in ("B", "G", "R", "U", "W") if c not in color_counts.Color.values
    ]
    missing_df = pd.DataFrame(
        [[c, 0] for c in missing_colors], columns=["Color", "Count"]
    )
    color_counts.append(missing_df)
    color_counts = color_counts.sort_values("Color")

    fig = go.Figure(
        data=[
            go.Bar(name=x[0], x=[x[0]], y=[x[1]], marker_color=color_hexes[i])
            for i, x in enumerate(color_counts.values)
        ],
    )
    fig.update_layout(
        title={
            "text": "Card Color Counts",
            "y": 0.9,
            "x": 0.5,
            "xanchor": "center",
            "yanchor": "top",
        }
    )
    return fig


def get_type_summary(inp_df: pd.DataFrame) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Get counts of the core card types and of the other keywords in your
    pool of drafted cards so far.
    """
    copied = inp_df.copy()
    card_types = [
        "Land",
        "Artifact",
        "Creature",
        "Enchantment",
        "Planeswalker",
        "Instant",
        "Sorcery",
    ]
    copied["keywords"] = copied["types"].apply(
        lambda x: [t.strip() for t in x.split(" ") if t.strip() not in ("", "-")]
    )
    vc = copied["keywords"].explode().value_counts().reset_index()
    main = (
        vc[vc["index"].apply(lambda x: x in card_types)]
        .copy()
        .rename(columns={"index": "type", "keywords": "count"})
    )
    other = (
        vc[~vc["index"].apply(lambda x: x in card_types)]
        .copy()
        .rename(columns={"index": "keyword", "keywords": "count"})
    )
    return main, other


def get_draft_level_info(
    alsa_tracker: ALSATrackerType, color_tracker: ColorTrackerType
):
    # handle the color tracking
    color_df = pd.DataFrame(
        [[k, v] for k, v in color_tracker.items()],
        columns=["color", "cards passed to you"],
    ).sort_values("color")

    # handle the ALSA tracking
    alsa_diff_sums = {"B": 0, "G": 0, "R": 0, "U": 0, "W": 0}
    for key, alsa_diff_dict in alsa_tracker.items():
        for inner_color, val in alsa_diff_dict.items():
            alsa_diff_sums[inner_color] += np.sum(val)
    alsa_df = pd.DataFrame(
        [[k, v] for k, v in alsa_diff_sums.items()],
        columns=["color", "ALSA difference sum"],
    )
    return color_df, alsa_df


def get_pool_summary(inp_df: pd.DataFrame):
    """
    Return all of the summary objects:
      - color count graph
      - type count df
      - keyword count df
    """
    color_count_graph = get_color_count_graph(inp_df)
    main_type_sum, other_type_sum = get_type_summary(inp_df)
    return color_count_graph, main_type_sum, other_type_sum


def add_format_to_cols(col_dicts: List[Dict[str, Any]]):
    for col_dict in col_dicts:
        if col_dict["name"] in percentage_cols:
            col_dict["format"] = Format(precision=1, scheme=Scheme.percentage)
            col_dict["type"] = "numeric"
        if col_dict["name"] in numeric_cols:
            col_dict["format"] = Format(precision=1, scheme=Scheme.fixed)
            col_dict["type"] = "numeric"
    return col_dicts


# every N seconds, our Interval fires, and we update most of the app
# by parsing the log
@app.callback(
    Output("live-update-text", "children"),
    Output("current-pick-table-holder", "children"),
    Output("draft-tracking-info", "children"),
    Output("pool-table-holder", "children"),
    Input("interval-component", "n_intervals"),
)
def update_metrics(n):
    """
    Update the whole app by parsing the log file
    """
    t0 = time.time()
    # create a 17Lands client with the API interaction removed
    # TODO: just fix __init__ to set these
    follower = DashFollower(token, "")
    follower.alsa_tracker = local_alsa_tracker
    follower.color_tracker = local_color_tracker
    follower.df = df
    follower.late_alsa_diff_threshold = LATE_ALSA_DIFF_THRESHOLD
    follower.multicolor_alsa_diff_factor = MULTICOLOR_ALSA_DIFF_FACTOR
    # parse your whole log
    r = follower.parse_log(logfile, follow=False, skip_bytes=SEEK_TO)
    t1 = time.time()
    print(f"parsed file in {t1 - t0} seconds")

    color_df, alsa_df = get_draft_level_info(local_alsa_tracker, local_color_tracker)

    style = {"padding": "5px", "fontSize": "16px"}
    # get the cards available to be picked
    picked_df = df.loc[getattr(follower, "pick_options", []), useful_cols].sort_values(
        "iwd", ascending=False
    )
    # get the cards you've already drafted
    pool_df = df.loc[getattr(follower, "pool_card_ids", []), useful_cols].sort_values(
        "iwd", ascending=False
    )
    # turn them both into Table objects
    picked_cols = [{"name": i, "id": i} for i in picked_df.columns]
    add_format_to_cols(picked_cols)
    pick_table = html.Div(
        dash_table.DataTable(
            id="pick_table",
            columns=picked_cols,
            data=picked_df.to_dict("records"),
            style_cell=cell_styling,
        ),
        style={"width": "80%", "margin": "auto"},
    )
    pool_table = html.Div(
        dash_table.DataTable(
            id="pool_table",
            columns=add_format_to_cols([{"name": i, "id": i} for i in pool_df.columns]),
            data=pool_df.to_dict("records"),
            style_cell=cell_styling,
        )
    )
    # get the summary statistics for your pool
    pool_summary, main_type_sum, other_type_sum = get_pool_summary(pool_df)
    main_type_sum = dash_table.DataTable(
        id="main_type_table",
        columns=[{"name": i, "id": i} for i in main_type_sum.columns],
        data=main_type_sum.to_dict("records"),
        style_cell=cell_styling,
    )
    other_type_sum = dash_table.DataTable(
        id="other_type_table",
        columns=[{"name": i, "id": i} for i in other_type_sum.columns],
        data=other_type_sum.to_dict("records"),
        style_cell=cell_styling,
    )
    # update the container with the summary info
    curr_div = dbc.Container(
        dbc.Row(
            [
                dbc.Col(pool_table, width={"size": 8}),
                dbc.Col(
                    [
                        dbc.Row(dcc.Graph(figure=pool_summary)),
                        dbc.Row(main_type_sum),
                        dbc.Row(html.Div("--")),
                        dbc.Row(other_type_sum),
                    ],
                    width={"size": 4},
                ),
            ]
        )
    )
    draft_info_div = dbc.Container(
        dbc.Row(
            [
                dbc.Col(
                    dash_table.DataTable(
                        columns=[{"name": i, "id": i} for i in color_df.columns],
                        data=color_df.to_dict("records"),
                        style_cell=cell_styling,
                    ),
                    width={"size": 4},
                ),
                dbc.Col(
                    dash_table.DataTable(
                        columns=[{"name": i, "id": i} for i in alsa_df.columns],
                        data=alsa_df.to_dict("records"),
                        style_cell=cell_styling,
                    ),
                    width={"size": 4},
                ),
            ]
        )
    )
    print(f"rebuilt dash components in {time.time() - t1} seconds")
    return [
        html.Span(
            f"Pack {follower.pack_number}, Pick {follower.pick_number}", style=style
        ),
        pick_table,
        draft_info_div,
        curr_div,
    ]


if __name__ == "__main__":
    app.run_server()